In [ ]:
#fix weird branch stuff
#download libraries
%pip install Pillow
%pip install tensorflow opencv-python matplotlib

In [ ]:
import tensorflow as tf
import os

In [ ]:

gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
print(tf.config.list_physical_devices('GPU'))

In [ ]:
#1: Get training data

#import images
from PIL import Image
from pathlib import Path

#count files in a folder
def count_files_in_folder(folder_path):
    folder = Path(folder_path)
    return sum(1 for item in folder.iterdir() if item.is_file())

#set folder path to l_training data
folder_path = 'data'
num_files = count_files_in_folder(folder_path)

imagelib = []

#store all files into library to use for data training
for i in range(num_files):
    imagelib.append("data/league_of_legends/img" + str(i + 1) +".png")

#verify correct number of files
print(len(imagelib))

#open image with line below
#image = Image.open(filename)

In [ ]:
#2 Load Data

import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#build data pipeline
data = tf.keras.utils.image_dataset_from_directory(folder_path)

In [ ]:
#access generator from data pipeline
#data_iterator = data.as_numpy_iterator()

In [ ]:
#resizes image to fit the batch, 
#batch = data_iterator.next()

In [ ]:
#show array of league of legends, vs destiny (we can tell by the sample of images that league is 1 and destiny is 0)
#class 1 = league
#class 0 = destiny
#batch[0] is all the images
#batch[1] are labels
#batch[1]

In [ ]:
#3 preprocess data
#applies the scaling while the data is being loaded
data = data.map(lambda x,y: (x/255, y))
scaled_iterator = data.as_numpy_iterator()
batch = scaled_iterator.next()

In [ ]:
batch[0].max()

In [ ]:
#visualize data with matplotlib
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img)
    ax[idx].title.set_text(batch[1][idx])

In [ ]:
len(data)

In [ ]:
#Split data into training, validation and testing
train_size = int(len(data)*.7)
val_size = int(len(data)*.2) + 1
test_size = int(len(data)*.1) + 1

In [ ]:
#ensure that values add up into the length of the batches
train_size+val_size+test_size

In [ ]:
#make sure data is shuffled before, in this case we already have shuffled the data with initial batch
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size + val_size).take(test_size)

In [ ]:
#Sequential models great if you have one data input and one data output
#functional api is better if you have multiple inputs and multiple outputs
from tensorflow.keras.models import Sequential

#Conv2d is a 2d convolution layer (spatial convolution over images)
#maxpooling2d condenses all the values in a region and returns max
#Flatten turns convolution into a format that dense can understand
#dropout for regularization
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
#Establish model
model = Sequential()

In [ ]:
#add layers
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))